In [1]:
!pip install langsmith > /dev/null
!pip install langchain > /dev/null
!pip install openai > /dev/null
!pip install python-dotenv > /dev/null

In [2]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm.invoke("How are you?")

AIMessage(content="I'm just a computer program, but I'm here and ready to help you! How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 11, 'total_tokens': 33, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-BwW03m1mpA7yDPYCLFl6V5PdjPFRK', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--ffdcd724-e96d-4f48-bc10-1b23b57d432b-0', usage_metadata={'input_tokens': 11, 'output_tokens': 22, 'total_tokens': 33, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Using Langsmith

In [4]:
from langchain.callbacks.tracers import LangChainTracer

tracer = LangChainTracer(project_name="My Project")
llm.invoke(input="How many people live in USA?", config = {"callbacks": [tracer]})

AIMessage(content="As of my last knowledge update in October 2023, the estimated population of the United States was around 334 million people. For the most current and accurate population figures, it's best to refer to the latest data from the U.S. Census Bureau or other reliable sources.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 14, 'total_tokens': 69, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-BwW1LEWtoq2bTIMO9CrEhcYTfkK7z', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--ce47485f-9ae9-44a6-a857-44d2c4d89fd7-0', usage_metadata={'input_tokens': 14, 'output_tokens': 55, 'total_tokens': 69, 'input_token_details': {'audio': 0, 'cache_read':

In [5]:
from langchain.callbacks import tracing_v2_enabled

with tracing_v2_enabled(project_name="My Project"):
    llm.invoke(input="How many people live in Germany?")

In [6]:
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0, tags=["my-llm-tag"])
prompt = PromptTemplate.from_template("Say {input}")
chain = LLMChain(llm=llm, prompt=prompt, tags=["one-tag", "another-tag"])

chain.invoke(input="Hello, World!", tags=["shared-tags"])

/var/folders/9l/6q0d47xn7p70rnjn04w1vvyw0000gn/T/ipykernel_4579/1826866266.py:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt, tags=["one-tag", "another-tag"])


{'input': 'Hello, World!', 'text': 'Hello, World!'}

In [7]:
from langchain.callbacks.manager import (
    trace_as_chain_group
)

with trace_as_chain_group("my_group_name") as group_manager:
    pass

from langchain_openai import ChatOpenAI
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["question"],
    template="What is the answer to {question}?",
)
chain = LLMChain(llm=llm, prompt=prompt)
with trace_as_chain_group("my_group") as group_manager:
    chain.invoke(input={"question": "What is your name?"}, callbacks=group_manager)
    chain.invoke(input={"question": "What is your name?"},  callbacks=group_manager)
    chain.invoke(input={"question": "What is your name?"}, callbacks=group_manager)

In [8]:
from langsmith import Client

client = Client()
project_runs = client.list_runs(project_name="default")
project_runs

<generator object Client.list_runs at 0x11c1689a0>

In [9]:
from datetime import datetime, timedelta

todays_runs = client.list_runs(
    project_name="default",
    start_time=datetime.now() - timedelta(days=1),
    run_type="llm",
)
todays_runs


<generator object Client.list_runs at 0x11c168b80>

In [10]:
for run in todays_runs:
    print(run)

id=UUID('91bc3ef6-0a9a-408a-8393-8976451cea18') name='ChatOpenAI' start_time=datetime.datetime(2025, 7, 23, 15, 56, 27, 183901) run_type='llm' end_time=datetime.datetime(2025, 7, 23, 15, 56, 27, 850806) extra={'invocation_params': {'model': 'gpt-3.5-turbo', 'model_name': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.9, '_type': 'openai-chat', 'stop': None}, 'options': {'stop': None}, 'batch_size': 1, 'metadata': {'ls_provider': 'openai', 'ls_model_name': 'gpt-3.5-turbo', 'ls_model_type': 'chat', 'ls_temperature': 0.9, 'revision_id': '86faf0a-dirty', 'ls_run_depth': 1}, 'runtime': {'sdk': 'langsmith-py', 'sdk_version': '0.4.6', 'library': 'langchain-core', 'platform': 'macOS-15.5-arm64-arm-64bit', 'runtime': 'python', 'py_implementation': 'CPython', 'runtime_version': '3.11.9', 'langchain_version': '0.3.26', 'langchain_core_version': '0.3.69', 'library_version': '0.3.69'}} error=None serialized=None events=[{'name': 'start', 'time': '2025-07-23T15:56:27.183901+00:00'}, {'name': 'en

### Metadata
Add metadata to filter runs later, for example for making an A/B test


In [16]:
from langchain_openai import ChatOpenAI
from langchain.chains.llm import LLMChain

chat_model = ChatOpenAI(model="gpt-4o-mini")
chain = LLMChain.from_string(llm=chat_model, template="What's the answer to {input}?")

chain.invoke({"input": "What is the meaning of life?"}, metadata={"source": "korean"})

{'input': 'What is the meaning of life?',
 'text': 'The meaning of life is a deeply philosophical question that has intrigued humanity for centuries, and answers can vary widely based on personal beliefs, cultural backgrounds, and philosophical perspectives. Some find meaning through relationships, love, and connections with others; others may find it in personal achievements, the pursuit of knowledge, or spiritual fulfillment.\n\nPhilosophers like Albert Camus suggest that life’s meaning is something we create ourselves in a seemingly indifferent universe. Religions often provide their own interpretations, offering frameworks for understanding existence and purpose through faith or adherence to certain moral principles.\n\nUltimately, many would argue that the meaning of life is a personal journey, inviting each individual to explore their own values, passions, and experiences to find what resonates most with them. What does it mean to you?'}

In [17]:
from langsmith import Client

client = Client()
runs = list(client.list_runs(
    project_name="default",
    filter='has(metadata, \'{"source": "korean"}\')',
))
print(list(runs))

[]


### Datasets

In [18]:
from langsmith import Client
client = Client()

dataset = client.create_dataset(dataset_name="testdataset", description="A dataset with key-value inputs and outputs")

In [19]:
client.create_example(
  inputs={
    "a-question": "What is the largest mammal?",
    "user-context": "The user is a 1st grader writing a bio report.",
  },
  outputs = {
    "answer": "The blue whale is the largest mammal.",
    "source": "https://en.wikipedia.org/wiki/Blue_whale",
  },
  dataset_id=dataset.id,
)

<class 'langsmith.schemas.Example'>(id=3156676e-2e36-4520-b3e2-5ae6e5e70637, dataset_id=133cc6d8-e2c0-4bdc-821a-25b390e9f7f8, link='https://smith.langchain.com/o/136ff41e-0adf-4e2e-84b5-de6e555e5349/datasets/133cc6d8-e2c0-4bdc-821a-25b390e9f7f8/e/3156676e-2e36-4520-b3e2-5ae6e5e70637')

In [20]:
dataset = client.upload_csv(
    csv_file="./extended_questions_answers.csv",
    input_keys=["Question"],
    output_keys=["Answer"],
    name="My CSV Dataset",
    description="Dataset created from a CSV file",
    data_type="kv",
)

Evaluation and Datasets

In [21]:
from langsmith import Client
from langchain.smith import RunEvalConfig, run_on_dataset

evaluation_config = RunEvalConfig(
    evaluators=[
        "qa",
        "context_qa",
        "cot_qa",
    ]
)

In [22]:
client = Client()
llm = ChatOpenAI(model="gpt-4o-mini")
run_on_dataset(
    dataset_name="My CSV Dataset",
    llm_or_chain_factory=llm,
    client=client,
    evaluation=evaluation_config,
    project_name="evalproject",
)

View the evaluation results for project 'evalproject' at:
https://smith.langchain.com/o/136ff41e-0adf-4e2e-84b5-de6e555e5349/datasets/5bf0496d-c8be-4a03-9ee2-1aeb280c36a1/compare?selectedSessions=a95adc45-6f7e-4c8b-ba34-da68e1e2eb58

View all tests for Dataset My CSV Dataset at:
https://smith.langchain.com/o/136ff41e-0adf-4e2e-84b5-de6e555e5349/datasets/5bf0496d-c8be-4a03-9ee2-1aeb280c36a1
[------------------------------------------------->] 5/5

{'project_name': 'evalproject',
 'results': {'6590d86c-f2b7-4ab6-93ea-2deb935904e8': {'input': {'Question': 'How is sushi rice different from regular rice?'},
   'feedback': [EvaluationResult(key='correctness', score=1, value='CORRECT', comment='CORRECT', correction=None, evaluator_info={'__run': RunInfo(run_id=UUID('8744f4fb-bfac-4071-90b1-c210b076bf3f'))}, feedback_config=None, source_run_id=None, target_run_id=None, extra=None),
    EvaluationResult(key='Contextual Accuracy', score=1, value='CORRECT', comment='CORRECT', correction=None, evaluator_info={'__run': RunInfo(run_id=UUID('3d4d79d8-d5fe-4390-8f87-58943e3cca95'))}, feedback_config=None, source_run_id=None, target_run_id=None, extra=None),
    EvaluationResult(key='COT Contextual Accuracy', score=1, value='CORRECT', comment="The student's answer is detailed and accurate. The student correctly identifies that sushi rice is different from regular rice in its type, preparation, and seasoning. The student also correctly explains 